*Z-Score:*
$$
Z = \frac{\bar{x_{1}}+\bar{x_{2}}}{\sqrt{\frac{s^{2}_{1}}{N_{1}} + \frac{s^{2}_{2}}{N_{2}}}}
$$


*$\chi^{2}$ Function:*
$$
\chi^{2} = \sum_{i}\frac{(y_{i} - y(h_{i}))^{2}}{\sigma ^{2}}
$$

*Correlation Coefficient:*
$$
r = \frac{\vec{A}\vec{B}}{|A||B|}
$$

*Potential Model:*
$$
R(h) = R_{1}exp[(-h1 - h)/H]
$$

In [2]:
#Import libraries
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [35]:
class location_data:
    def __init__(self,location,altitude,data,type = None):
        self.location = location
        self.altitude = altitude
        self.data = data
        self.type = type


class comparison_data:
    def __init__(self,location_data_1,location_data_2,Z_score,Chi,r):
        self.location_data_1 = location_data_1
        self.location_data_2 = location_data_2
        self.Z_score = Z_score
        self.Chi = Chi
        self.r = r

    def export_format(self):
        export_array = [
            [f"{self.location_data_1.location}",f"{self.location_data_2.location}"],
            [f"{self.location_data_1.altitude}",f"{self.location_data_2.altitude}"],
            ["============================","============================"],
            ["Test","Value"],
            ["Z-Score",f"{self.Z_score:.2f}"],
            ["Chi Sqrd",f"{self.Chi}"]
            ["Correlation Coefficient", f"{self.r}"]
            ]
        return export_array
        

<>:24: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:24: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\slick\AppData\Local\Temp\ipykernel_827656\3130029225.py:24: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  ["Chi Sqrd",f"{self.Chi}"]


In [39]:
#Open and read from file
def read_from_file(file_name):
    """Function reads our data from our file and transforms it into
    a numpy array, which we return.
    Parameters: 
      file_name
    Return: 
      file_data as a numpy array
    """
    data = pd.read_csv(file_name,encoding='unicode_escape')
    file_data = data.to_numpy()
    return file_data

#Parse through and return list of location data
def parse_file_data(file_data):
    """ Function parses file_data and splits it into classes, then appends
    it to a list.
    Parameters:
        file_data (numpy array)
    Return:
        list of location_data objects
    """
    #Create new blank list
    location_data_list = []

    for fdata in file_data:
        #Parse out our variables
        print(fdata)
        location = str(fdata[1])
        print(location)
        altitude = str(fdata[2])
        print(altitude)
        data = np.array(fdata[3:]).astype(float)
        print(data[0])

        #Construct class and append to list
        local_data = location_data(location,altitude,data)
        location_data_list.append(local_data)
    return location_data_list

def compare_data(location_data_list):
    """Function takes the list of location_data and parses through 
    and compares it against the other values.
    Parameters:
        List of location_data objects
    Returns:
        uhhhhhhhhh...... 
    """
    compare_data_list = []

    for location_data_1 in location_data_list:
        x1_array = location_data_1.data
        print(x1_array)

        for location_data_2 in location_data_list:
            x2_array = location_data_2.data

            z_score = ZScore(x1_array,x2_array)
            chi_sqrd = Chi_sqrd(x1_array,x2_array)
            r = correlation_coefficient(x1_array,x2_array)

            compared_data = comparison_data(location_data_1,location_data_2,z_score,chi_sqrd,r)
            compare_data_list.append(compared_data)
    


def mean(x_array):
    N = len(x_array)
    return sum(x_array)/N

def stdev(x_array):
    x_mean = mean(x_array)
    stdev_x = 0
    N = len(x_array)
    for x in x_array:
        stdev_x += (x-x_mean)**2
    return np.sqrt((stdev_x)/(N-1))

#Def Zscore test
def ZScore(x1_array,x2_array):
    #Calculate the number of values in both arrays
    N1 = len(x1_array)
    N2 = len(x2_array)

    #Calculate the mean of both arrays
    x1_mean = mean(x1_array)
    x2_mean = mean(x2_array)

    #Calculate the Stdev of both arrays
    stdev_x1 = stdev(x1_array)
    stdev_x2 = stdev(x2_array)

    #Calculate and return our Z-Score
    return (x1_mean + x2_mean)/(np.sqrt((stdev_x1*stdev_x1/N1) + (stdev_x2 * stdev_x2/N2)))

def correlation_coefficient(x1_array,x2_array):
    #Define A and B
    A = x1_array - mean(x1_array)
    B = x2_array - mean(x2_array)

    #Find our magnitudes for A and B
    Amag = np.linalg.norm(A)
    Bmag = np.linalg.norm(B)

    #Calculate and return our correlation coefficient
    r = A@B/(Amag*Bmag)
    return r

def Chi_sqrd(x1_array,x2_array,sigma):
    # Calculate our Chi squared value
    chi_sqrd = sum((x1_array-x2_array)*(x1_array-x2_array)/sigma/sigma)
    return chi_sqrd

def main():
    #Pull data from file and parse it into classes
    file_name = "test_data.csv"
    file_data = read_from_file(file_name)
    location_data_list = parse_file_data(file_data)
    compare_data(location_data_list)

    

    return

main()

['Romney' 'Romney']
Romney


IndexError: index 2 is out of bounds for axis 0 with size 2